# Lesson 12: EDA

In [3]:
import logging
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
logger = logging.getLogger ()
logger.setLevel(logging.INFO)

In [4]:
url = ('https://gist.githubusercontent.com/teamtom/' + 
       '1af7b484954b2d4b7e981ea3e7a27f24/raw/' + 
       '114fb69dce56b4462a9c3a417e7402330616ad4f/' + 
       'titanic_full.csv')
df_raw = pd.read_csv(url)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  746 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [5]:
df_raw.sample(10, random_state=2025)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
308,3,0,Cribb Mr. John Hatfield,male,44.0,0,1,371362,16.1000,NaN,S,NaN,NaN,Bournemouth England Newark NJ
311,3,0,Badt Mr. Mohamed,male,40.0,0,0,2623,7.2250,NaN,C,NaN,NaN,NaN
391,3,0,Andreasson Mr. Paul Edvin,male,20.0,0,0,347466,7.8542,NaN,S,NaN,NaN,Sweden Chicago IL
344,3,1,Asplund Miss. Lillian Gertrud,female,5.0,4,2,347077,31.3875,NaN,S,15,NaN,Sweden Worcester MA
1189,1,1,Madill Miss. Georgette Alexandra,female,15.0,0,1,24160,211.3375,B5,S,2,NaN,St Louis MO
1221,2,0,Pain Dr. Alfred,male,23.0,0,0,244278,10.5000,NaN,S,NaN,NaN,Hamilton ON
898,1,0,Baumann Mr. John D,male,NaN,0,0,PC 17318,25.9250,NaN,S,NaN,NaN,New York NY
1106,1,1,Aubart Mme. Leontine Pauline,female,24.0,0,0,PC 17477,69.3000,B35,C,9,NaN,Paris France
461,3,0,O'Donoghue Ms. Bridget,female,NaN,0,0,364856,7.7500,NaN,Q,NaN,NaN,NaN
1027,3,0,Novel Mr. Mansouer,male,28.5,0,0,2697,7.2292,NaN,C,NaN,181.0,NaN


### 2. Data Pre-processing
**Objectives**: Perform the data cleaning, data transformation, and data reduction steps to avoid the data mismatching, noisy data or data not wrangled

In [6]:
df_baking = df_raw.copy()
df_baking = df_raw.drop(["boat", "body", "home.dest"], axis=1) # remove unccessary columns
df_baking["pclass" ] = df_baking ['pclass'].replace({1: 'A', 2: 'B', 3: 'C'}) # convert int's to strings/objects
df_baking['pclass'] = df_baking['pclass'].astype('category') # convert strings to category
df_baking['survived'] = df_baking['survived'].replace({1: 'Yes', 2:'No'})
df_baking['survived'] = df_baking['survived'].astype('category')
df_baking['sex'] = df_baking['sex'].astype('category')
df_baking['cabin'] = df_baking['cabin'].astype('category')
df_baking['embarked'] = df_baking['embarked'].astype('category')
df_baking


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,C,0,Mahon Miss. Bridget Delia,female,NaN,0,0,330924,7.8792,NaN,Q
1,A,0,Clifford Mr. George Quincy,male,NaN,0,0,110465,52.0000,A14,S
2,C,0,Yasbeck Mr. Antoni,male,27.0,1,0,2659,14.4542,NaN,C
3,C,Yes,Tenglin Mr. Gunnar Isidor,male,25.0,0,0,350033,7.7958,NaN,S
4,C,0,Kelly Mr. James,male,34.5,0,0,330911,7.8292,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...
1304,C,0,Rice Master. Eugene,male,2.0,4,1,382652,29.1250,NaN,Q
1305,C,0,Dika Mr. Mirko,male,17.0,0,0,349232,7.8958,NaN,S
1306,C,0,Palsson Miss. Torborg Danira,female,8.0,3,1,349909,21.0750,NaN,S
1307,A,0,Widener Mr. George Dunton,male,50.0,1,1,113503,211.5000,C80,C


In [7]:
 df_baking['name'].nunique()

1307

In [8]:
 df_baking['ticket'].nunique()

929

In [9]:
 df_baking['cabin'].nunique()

186

In [11]:
for index, i in enumerate(df_baking['cabin']):
    if pd.isna(i):
        df_baking.loc[index, 'cabin'] = 0
    else:
        df_baking.loc[index, 'cabin'] = 1


TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first